In [3]:
import torch
import torchvision
import torch.nn as nn                                              
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(linewidth = 120)
torch.set_grad_enabled(True)
import torchvision
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pyautogui
from PIL import Image 
from Xlib import display
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 7, 5)
        self.pool = nn.MaxPool2d(3, 3)
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(7, 10, 5)
        self.fc1 = nn.Linear(280, 90)
        self.fc2 = nn.Linear(90, 30)
        self.fc3 = nn.Linear(30, 1)

    def forward(self, x):
        x = self.dropout(self.pool(F.relu(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

drunk_driver = Net()
print(drunk_driver)
drunk_driver = torch.load("torch/alpha.pth")

Net(
  (conv1): Conv2d(3, 7, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(7, 10, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=280, out_features=90, bias=True)
  (fc2): Linear(in_features=90, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=1, bias=True)
)


In [30]:
def preprocess_live(img_obj, show = False):
    
    np_im = np.array(img_obj)
    if show == True:
        plt.imshow(np_im)
        plt.show()
    np_im = np.moveaxis(np_im, 2,0)
    inp = torch.from_numpy(np_im).unsqueeze(0).float()  ## tensor ready to be fed to nn 
    return inp 

def image_to_result(img_path):
    
    im_tensor = preprocess(img_path, show = True)
    res = drunk_driver(im_tensor).item()
    
    print(res)
    
def capture_resize_to_pil(co_ord, size = (100,100)):
    
    im = pyautogui.screenshot(region = co_ord ).resize((size), Image.ANTIALIAS)
    return im

def make_a_decision(image):  ## image to steer  val 
    
    im_tensor = preprocess_live(image, show = False)
    res = drunk_driver(im_tensor).item()
    return res

def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]

def move_mouse_co_ord_x(x_val , time = 0.001):
    
    current_y = find_mouse(y = True)
    pyautogui.moveTo(x_val, current_y, duration= time )

In [1]:
def seatbelts():
    
    foo = cv2.imread("r.png")
    cv2.imshow('image',foo)
    
    try :
        while True:

                foo = capture_resize_to_pil((0,90,800,560), size = (80,56))   ## template inp
                steer = make_a_decision(foo)

                move_mouse_co_ord_x(x_co , time = 0)
    except KeyboardInterrupt:

        print ("end")
   

In [ ]:
seatbelts()